## Importamos las librerias necesarias

In [1]:
#Importamos pandas, numpy
import numpy as np
import pandas as pd

## Importamos el dataset limpio para cargarlo en un dataframe

In [2]:
#Guardamos el path del archivo en una variable para despues cargarlo en un df

file = 'C:/Users/Ramiro/Desktop/Digital House/Desafios/Desafio 2/dataset limpio.csv'

# Creamos un DataFrame con la info del csv
df = pd.read_csv(file)

### Dejamos solo las columnas necesarias

In [3]:
df = df[['Tipo_Propiedad', 'Barrio','Provincia', 'Precio', 'Precio_ARS', 'Precio_Dolares', 'Metros_Totales',
       'Metros_Cubiertos', 'Precio_m2_Dls', 'Precio_m2', 'Cantidad_Ambientes','Propiedad_Especial']]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 12 columns):
Tipo_Propiedad        121220 non-null object
Barrio                121197 non-null object
Provincia             121220 non-null object
Precio                100810 non-null float64
Precio_ARS            97850 non-null float64
Precio_Dolares        97389 non-null float64
Metros_Totales        108336 non-null float64
Metros_Cubiertos      108865 non-null float64
Precio_m2_Dls         67754 non-null float64
Precio_m2             87371 non-null float64
Cantidad_Ambientes    86078 non-null float64
Propiedad_Especial    121220 non-null float64
dtypes: float64(9), object(3)
memory usage: 11.1+ MB


### Antes de eliminar observaciones, deberiamos chequear la distribucion por Provincias para asegurarnos que no nos queden zonas sin observaciones

In [5]:
df['Provincia'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

Vemos que en el dataset original existen provincias con muy pocas observaciones. Esto generara que el modelo no pueda predecir correctamente, o del todo, esas provincias.

### Nos quedamos solo con las observaciones que no tienen valores nulos

Prodiamos intentar predecir cualquiera de las variables de precio (en dolares o pesos). Para evaluar que tan bien performan los modelos, vamos a intentar predecir tanto el Precio total en pesos (Precio_ARS) y el Precio por metro cuadrado en pesos (Precio_m2)

In [6]:
df_precio = df.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

df_m2 = df.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

In [7]:
df_precio = df_precio.dropna(axis =0 , how = 'any')
df_m2 = df_m2.dropna(axis =0 , how = 'any')

In [8]:
#Dataset Precio total
df_precio['Provincia'].value_counts()

Capital Federal                 21400
Bs.As. G.B.A. Zona Norte        15078
Buenos Aires Costa Atlántica     7242
Bs.As. G.B.A. Zona Sur           6450
Bs.As. G.B.A. Zona Oeste         5315
Córdoba                          4606
Santa Fe                         2874
Buenos Aires Interior            1098
Corrientes                        308
Mendoza                           288
Río Negro                         257
Neuquén                           223
Misiones                          216
Tucumán                           103
Chubut                             97
San Luis                           79
Entre Ríos                         58
Salta                              57
Tierra Del Fuego                   24
Chaco                              16
Santa Cruz                         10
La Pampa                            9
Jujuy                               6
Catamarca                           4
San Juan                            3
La Rioja                            2
Formosa     

In [9]:
#Dataset Precio m2
df_m2['Provincia'].value_counts()

Capital Federal                 21418
Bs.As. G.B.A. Zona Norte        15065
Buenos Aires Costa Atlántica     6208
Bs.As. G.B.A. Zona Sur           6086
Bs.As. G.B.A. Zona Oeste         5094
Córdoba                          4428
Santa Fe                         2711
Buenos Aires Interior            1042
Corrientes                        288
Mendoza                           285
Neuquén                           209
Río Negro                         176
Misiones                          165
San Luis                           76
Tucumán                            65
Entre Ríos                         53
Salta                              53
Chubut                             32
Tierra Del Fuego                   24
Chaco                              15
La Pampa                            9
Catamarca                           7
Santa Cruz                          7
Jujuy                               4
San Juan                            3
La Rioja                            2
Santiago Del

Efectivamente, notamos que al momento de quedarnos solo con las observaciones con todos los valores completos, perdimos Provincias enteras como Formosa, Catamarca y Jujuy.
Esto va a causar que al momento de querer realizar una prediccion sobre una propiedad de esa provincia, el modelo prediga incorrectamente.
Entendemos que es un error a tener en cuenta. La forma de poder corregirlo seria agregar observaciones al dataset.

## Modelado

Para comenzar el modelado, debemos transformar las variables categoricas en dummies, luego normalizaremos las variables para facilitar la corrida de los distintos modelos.
Vamos a modelar primero tratando predecir el Precio total para todo el pais, y luego el precio por metro cuadrado.

Finalmente, generaremos modelos mas acotados (a ciertas provincias o zonas) buscando una performance mejor de los modelos

In [10]:
#Creamos variables dummies de las variables categoricas

#Provincia
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Provincia'], prefix= 'prov')], axis = 1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Provincia'], prefix= 'prov')], axis = 1)

#Barrio
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Barrio'], prefix= 'barrio')], axis =1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

In [11]:
#Dropeo de las columnas originales
df_precio = df_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
df_m2 = df_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

#### Importamos todas las librerias necesarias para el modelado

In [12]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

#### Armamos nuestro vector objetivo y nuestra matriz de features

In [13]:
#Separamos nuestra matriz de features

X = df_precio.drop(['Precio_ARS'], axis = 1)
y = df_precio['Precio_ARS']

#### Separamos los datos en sets de testeo y training

In [ ]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

### Regresion Lineal

In [15]:
#Instanciamos el modelo

lm = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal = lm.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio = reg_lineal.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal.score(X_test, y_test))

Score del Regresion Lineal: 0.576628765212965


#### Como predice el mismo modelo el precio por metro cuadrado?

In [35]:
#Separamos nuestra matriz de features

X_m2 = df_m2.drop(['Precio_m2'], axis = 1)
y_m2 = df_m2['Precio_m2']

#Separamos en train y test
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(X_m2, y_m2, test_size=0.10, random_state=0)

In [107]:
#Instanciamos el modelo

lm_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_m2 = lm_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_m2 = reg_lineal_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_m2.score(X_test_m2, y_test_m2))

Score del Regresion Lineal: -22827893931134.164


## Meter un grafico aca comparando los dos scores???

Vemos que hay una diferencia considerable entre el score del modelo que predice el precio total, que el que predice el precio por metro cuadrado. Tengamos en cuenta que en ninguno de los dos modelos incluimos la otra variable al momento de modelar, ya que entendemos que por ejemplo, el precio por metro cuadrado explicaria casi el total de la variabilidad del Precio Total de la propiedad

El modelo de regresion lineal multiple es demasiado simple para poder obtener un insight lo suficientemente convincente, por lo cual vamos a aplicar dos modelos que incluyen tecnicas de regularizacion: Ridge y Lasso

### Primero debemos normalizar las features.
Las variables dummies no es necesario que sean normalizadas ya que son variables dicotomicas (0 o 1).

Recordemos que nuestros sets de training y testeo quedaron definidos de la siguiente manera:

`X_train, X_test, y_train, y_test`: Para el modelar el Precio_ARS

`X_train_m2, X_test_m2, y_train_m2, y_test_m2`: Para modelar el Precio_m2

In [18]:
#Importamos StandardScaler para normalizar las variables
from sklearn.preprocessing import StandardScaler
scaler_precio = StandardScaler()
X_train = scaler_precio.fit_transform(X_train)

In [19]:
print('media:',np.mean(X_train[:,1]))
print('desvio:',np.std(X_train[:,1]))

media: -6.451563387304831e-17
desvio: 0.9999999999999999


# xq no da 0 y 1 exactamente???

## Regresion Lineal Ridge

In [20]:
#Instanciamos el modelo

lm_ridge =  linear_model.RidgeCV(alphas=[0.1, 20, 100], cv=5)

#Entrenamos el modelo
reg_lineal_ridge = lm_ridge.fit(X_train, y_train)
score_ridge_precio = reg_lineal_ridge.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Ridge:', reg_lineal_ridge.score(X_test, y_test))

Score del modelo Ridge: -7741.4053105149815


In [21]:
reg_lineal_ridge.alpha_

20

# REVISAR

### Como funciona el modelo para el precio por metro cuadrado?

In [22]:
#Importamos StandardScaler para normalizar las variables
scaler_m2 = StandardScaler()
X_train_m2 = scaler_m2.fit_transform(X_train_m2)

In [23]:
print('media:',np.mean(X_train_m2[:,1]))
print('desvio:',np.std(X_train_m2[:,1]))

media: -3.579259856066326e-18
desvio: 1.0


In [24]:
#Instanciamos el modelo
lm_ridge_m2 =  linear_model.RidgeCV(alphas=[10, 100, 10], cv=5)

#Entrenamos el modelo
reg_lineal_ridge_m2 = lm_ridge_m2.fit(X_train_m2, y_train_m2)
score_ridge_m2 = reg_lineal_ridge_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo
print('Score del modelo Ridge:', reg_lineal_ridge_m2.score(X_test_m2, y_test_m2))

Score del modelo Ridge: -301.5904675969028


In [25]:
reg_lineal_ridge.alpha_


20

## Regresion Lasso

In [42]:
#Instanciamos el modelo
lm_lasso =  linear_model.LassoCV(alphas=[1000, 100000, 100], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso = lm_lasso.fit(X_train, y_train)

print('Score del modelo Lasso:', reg_lineal_lasso.score(X_test, y_test))

C:\Users\Ramiro\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Score del modelo Lasso: -7752.909902068187


### Como funciona el modelo para el precio por metro cuadrado?

In [43]:
#Instanciamos el modelo
lm_lasso_m2 =  linear_model.LassoCV(alphas=[1000, 100000, 100], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso_m2 = lm_lasso_m2.fit(X_train_m2, y_train_m2)

print('Score del modelo Lasso:', reg_lineal_lasso_m2.score(X_test_m2, y_test_m2))

Score del modelo Lasso: 0.1368339358570061


## Elastic Net

In [48]:
#Instanciamos el modelo

lm_elastic_net =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1],cv = 5)

#Entrenamos el modelo
elastic_net = lm_elastic_net.fit(X_train, y_train)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net.score(X_test, y_test))

Score del modelo Elastic Net: -7738.299211283508


### Como funciona el modelo para el precio por metro cuadrado?

In [49]:
#Instanciamos el modelo

lm_elastic_net_m2 =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1], alphas = [10, 100, 10],cv = 5)

#Entrenamos el modelo
elastic_net_m2 = lm_elastic_net_m2.fit(X_train_m2, y_train_m2)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net_m2.score(X_test_m2, y_test_m2))

Score del modelo Elastic Net: 0.2010685045740972


In [75]:
df['Provincia'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

# Modelo para Capital y Buenos Aires

In [91]:
df_BsAs = df.loc[df['Provincia'].isin(['Capital Federal','Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Sur',\
                             'Buenos Aires Costa Atlántica','Bs.As. G.B.A. Zona Oeste','Buenos Aires Interior'])]


In [97]:
BsAs_precio = df_BsAs.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

BsAs_m2 = df_BsAs.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

In [98]:
BsAs_precio = BsAs_precio.dropna(axis =0 , how = 'any')
BsAs_m2 = BsAs_m2.dropna(axis =0 , how = 'any')

In [99]:
#Creamos variables dummies de las variables categoricas

#Provincia
BsAs_precio = pd.concat([BsAs_precio, pd.get_dummies(data = BsAs_precio['Provincia'], prefix= 'prov')], axis = 1)
BsAs_m2 = pd.concat([BsAs_m2, pd.get_dummies(data = BsAs_m2['Provincia'], prefix= 'prov')], axis = 1)

#Barrio
BsAs_precio = pd.concat([BsAs_precio, pd.get_dummies(data = BsAs_precio['Barrio'], prefix= 'barrio')], axis =1)
BsAs_m2 = pd.concat([BsAs_m2, pd.get_dummies(data = BsAs_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
BsAs_precio = pd.concat([BsAs_precio, pd.get_dummies(data = BsAs_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
BsAs_m2 = pd.concat([BsAs_m2, pd.get_dummies(data = BsAs_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

In [103]:
#Dropeo de las columnas originales
BsAs_precio = BsAs_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
BsAs_m2 = BsAs_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

In [104]:
#Separamos nuestra matriz de features

X_bsas = BsAs_precio.drop(['Precio_ARS'], axis = 1)
y_bsas = BsAs_precio['Precio_ARS']

#### Separamos los datos en sets de testeo y training

In [105]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_bsas, y_bsas, test_size=0.10, random_state=1)

### Regresion Lineal

In [106]:
#Instanciamos el modelo

lm_bsas = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_bsas = lm_bsas.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio_bsas = reg_lineal_bsas.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_bsas.score(X_test, y_test))

Score del Regresion Lineal: -235741889980637.03


#### Como predice el mismo modelo el precio por metro cuadrado?

In [125]:
#Separamos nuestra matriz de features

X_bsas_m2 = BsAs_m2.drop(['Precio_m2'], axis = 1)
y_bsas_m2 = BsAs_m2['Precio_m2']

#Separamos en train y test
X_train_bsas_m2, X_test_bsas_m2, y_train_bsas_m2, y_test_bsas_m2 = train_test_split(X_bsas_m2, y_bsas_m2, test_size=0.10, random_state=0)

In [129]:
print(X_bsas_m2.shape, y_bsas_m2.shape)

(54913, 642) (54913,)


In [128]:
#Instanciamos el modelo

lm_bsas_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_bsas_m2 = lm_bsas_m2.fit(X_train_bsas_m2, y_train_bsas_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_bsas_m2 = reg_lineal_bsas_m2.score(X_test_bsas_m2, y_test_bsas_m2)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_m2.score(X_test_bsas_m2, y_test_bsas_m2))

ValueError: shapes (5492,642) and (872,) not aligned: 642 (dim 1) != 872 (dim 0)